In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer,AutoModelForSequenceClassification,RobertaForSequenceClassification,RobertaModel

In [2]:
train_df=pd.read_csv("/data/linqinhong/zindi/vaccine/data/Train.csv")
test_df=pd.read_csv("/data/linqinhong/zindi/vaccine/data/Test.csv")

In [3]:
train_df.head(5)

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [4]:
na_list=list(train_df[train_df["label"].isnull()==True].index)

In [5]:
train_df.drop(na_list,axis=0,inplace=True)

In [6]:
train_df[train_df["label"].isnull()==True]

,tweet_id,safe_text,label,agreement


In [7]:
train_df.drop("tweet_id",inplace=True,axis=1)

In [8]:
train_df.head(5)

,safe_text,label,agreement
0,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,I mean if they immunize my kid with something ...,-1.0,1.0
4,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [9]:
train_df["safe_text"]=train_df["safe_text"].apply(lambda x:x.replace("&amp",""))

删除带有#的，不一定能提高表现，需要对比

In [10]:
import re
pattern1=re.compile(r"#[\w]*")

In [11]:
train_df["safe_text"]=train_df["safe_text"].apply(lambda x:re.sub(pattern1,"",x))

In [12]:
train_df.head()

,safe_text,label,agreement
0,Me ; The Big Homie meanboy3000 @ Stegman ...,0.0,1.0
1,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,"VACCINES, DO NOT VACCINATE YOUR CHILD",-1.0,1.0
3,I mean if they immunize my kid with something ...,-1.0,1.0
4,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


删除带有<>符号里的内容

In [13]:
pattern2=re.compile(r"<[\w]*>")
train_df["safe_text"]=train_df["safe_text"].apply(lambda x:re.sub(pattern2,"",x))

In [14]:
train_df.head()

,safe_text,label,agreement
0,Me ; The Big Homie meanboy3000 @ Stegman ...,0.0,1.0
1,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,"VACCINES, DO NOT VACCINATE YOUR CHILD",-1.0,1.0
3,I mean if they immunize my kid with something ...,-1.0,1.0
4,Thanks to Catch me performing at La Nuit NYC ...,0.0,1.0


In [15]:
train_df["safe_text"]=train_df["safe_text"].apply(str)

In [16]:
train_df["safe_text"]=train_df["safe_text"].apply(str.lower)

In [17]:
train_df.head()

,safe_text,label,agreement
0,me ; the big homie meanboy3000 @ stegman ...,0.0,1.0
1,i'm 100% thinking of devoting my career to pro...,1.0,1.0
2,"vaccines, do not vaccinate your child",-1.0,1.0
3,i mean if they immunize my kid with something ...,-1.0,1.0
4,thanks to catch me performing at la nuit nyc ...,0.0,1.0


In [18]:
from nltk.corpus import stopwords
stopword=stopwords.words("english")

In [19]:
import contractions
train_df["safe_text"]=train_df["safe_text"].apply(contractions.fix)

In [20]:
def remove_stopword(text):
    return " ".join(i for i in text.split() if i not in stopword)

In [21]:
train_df["safe_text"]=train_df["safe_text"].apply(remove_stopword)

In [22]:
train_df["label"]=train_df["label"].apply(int)

In [23]:
train_df.reset_index(drop=True,inplace=True)

In [24]:
modelname="cardiffnlp/twitter-roberta-base-sentiment"

In [25]:
tokenizer=AutoTokenizer.from_pretrained(modelname)

In [26]:
model=RobertaModel.from_pretrained(modelname)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment were not used when initializing RobertaModel: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

In [27]:
from datasets import Dataset
train_dataset=Dataset.from_pandas(train_df)
test_dataset=Dataset.from_pandas(test_df)

In [28]:
def tokenize(text):
    result=tokenizer(text["safe_text"])
    return result

In [29]:
train_dataset

Dataset({
    features: ['safe_text', 'label', 'agreement'],
    num_rows: 10000
})

In [30]:
test_dataset

Dataset({
    features: ['tweet_id', 'safe_text'],
    num_rows: 5177
})

In [31]:
train_dataset=train_dataset.map(tokenize,batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [32]:
train_dataset["input_ids"][0]

[0, 131, 380, 9486, 324, 1266, 9902, 31640, 787, 1690, 3733, 397, 1690, 4, 2]

In [33]:
train_dataset=train_dataset.remove_columns("safe_text")

In [34]:
train_dataset.column_names

['label', 'agreement', 'input_ids', 'attention_mask']

In [35]:
train_dataset=train_dataset.rename_column("label","labels")

In [36]:
def process_labels(text):
    return {"labels":text["labels"]+1}

In [37]:
train_dataset=train_dataset.map(process_labels)

  0%|          | 0/10000 [00:00<?, ?ex/s]

In [38]:
train_dataset=train_dataset.remove_columns("agreement")

In [39]:
train_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [40]:
from sklearn.model_selection import StratifiedKFold,KFold
num_split=5
seed=3407
#SKF=StratifiedKFold(n_splits=num_split,shuffle=True)

In [41]:
import torch
device=torch.device("cuda") if torch.cuda.is_available()==True else torch.device("cpu")
model.to(device)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [42]:
from transformers import DataCollatorWithPadding
datacollator=DataCollatorWithPadding(tokenizer=tokenizer)

In [43]:
from torch.optim import AdamW
from transformers import get_scheduler
lr_rate=0.0005
optimizer=AdamW(model.parameters(),lr=lr_rate)
#lr_scheduler=get_scheduler('linear',optimizer=optimizer,num_warmup_steps=20,num_training_steps=train_steps)

In [45]:
from torch.utils.data import DataLoader
from torch.nn.modules.loss import  CrossEntropyLoss
import evaluate
from torch import softmax
metric_train=evaluate.load("accuracy")
metric_val=evaluate.load("accuracy")
epoches=50
total_loss=[]
from tqdm.auto import tqdm
epoch_bar=tqdm(range(epoches))
loss_fn=CrossEntropyLoss()
model.eval()
for epoch in range(epoches):
    KF = KFold(n_splits=num_split, shuffle=True,random_state=seed+epoch)
    split_result=KF.split(train_dataset,train_dataset["labels"])
    for train_split,val_split in split_result:
        train_split_data=train_dataset.select(train_split)#.remove_columns("labels")
        #train_split_label=train_dataset.select(train_split)["labels"]
        val_split_data=train_dataset.select(val_split)#.remove_columns("labels")
        #val_split_labels=
        train_dataloader=DataLoader(train_split_data,shuffle=True,batch_size=64,collate_fn=datacollator)
        val_dataloader=DataLoader(val_split_data,shuffle=True,batch_size=16,collate_fn=datacollator)
        train_steps=len(train_dataloader)
        lr_scheduler=get_scheduler('linear',optimizer=optimizer,num_warmup_steps=0.2*train_steps,num_training_steps=train_steps)
        train_bar=tqdm(range(len(train_dataloader)))
        #val_bar=tqdm(range(len(val_dataloader)))
        Loss=0
        for batch in train_dataloader:
            batch={k:v.to(device) for k,v in batch.items()}
            optimizer.zero_grad()
            output=model(input_ids=batch["input_ids"],attention_mask=batch["attention_mask"])
            labels=batch["labels"]
            break
            loss=loss_fn(class_output,labels)
            Loss+=loss.item()
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            train_pred=torch.argmax(class_output,dim=1)
            metric_train.add_batch(predictions=train_pred,references=batch["labels"])
            train_acc=metric_train.compute()
            train_bar.update(1)
        print(train_acc)
        total_loss.append(Loss/len(train_dataloader))
            #break

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

NameError: name 'train_acc' is not defined

In [54]:
output[0].shape

TypeError: softmax() received an invalid combination of arguments - got (numpy.ndarray), but expected one of:
 * (Tensor input, name dim, *, torch.dtype dtype)
 * (Tensor input, int dim, torch.dtype dtype)


In [ ]:
for i in Classifier.linear1.parameters():
    print(i)

In [ ]:
output1=model(input_ids=batch["input_ids"],attention_mask=batch["attention_mask"])

In [ ]:
for i in model.parameters():
    print(i)

In [ ]:
B=Classifier(output1.pooler_output)

In [23]:
from torch import softmax
import torch
import numpy as np

In [29]:
A=torch.tensor([0,1],dtype=torch.float,requires_grad=True)

In [35]:
A=softmax(A,dim=0)

In [36]:
A

tensor([0.4435, 0.5565], grad_fn=<SoftmaxBackward>)

In [ ]:
model1=RobertaForSequenceClassification.from_pretrained(modelname)

In [ ]:
model1.classifier

In [ ]:
output.pooler_output

In [ ]:
A=linear(output.pooler_output)

In [ ]:
from torch.nn import CrossEntropyLoss
from torch import softmax
result=softmax(A,dim=0)

In [ ]:
aa=train_dataset.select(train_split)["labels"]

In [ ]:
len(aa)

In [ ]:
result[0].shape

In [ ]:
torch.tensor(aa[0:32]).to(device).shape

In [ ]:
loss_fn=CrossEntropyLoss()
loss=loss_fn(result[1:2],torch.tensor(aa[1:2],dtype=torch.long).to(device))

In [ ]:
loss

In [ ]:
from torch.nn import Linear
from torch.nn import CrossEntropyLoss
from torch import relu
from torch import softmax
Linear=Linear(1024,3).to(device)
loss_fn=CrossEntropyLoss()
def classifier(input,labels):
    loss=0
    for out,label in zip(output.pooler_output,labels):
        mid=Linear(out)
        predict=softmax(relu(mid))
        loss+=loss_fn(predict,labels)
    return loss

In [ ]:
output.pooler_output

In [ ]:
train_dataset.select(train_split)["labels"]

In [ ]:
classifier(output.pooler_output.items.detach(),train_dataset.select(train_split)["labels"])

In [ ]:
output.pooler_output